# From xml to json

In [ ]:
__author__ = 'Qiao Jin'
import os
import glob
import json
import xml.etree.ElementTree as ET

files = os.listdir("pubmed_2022")
files.sort()
for xml_path in files:
    
    check = os.path.exists("pubmed_2022d/%s" % xml_path.replace("xml", "json"))
    if not check:
        print('Processing %s' % xml_path)
        output = {}

        tree = ET.parse("pubmed_2022/%s" % xml_path)
        root = tree.getroot()

        for citation in root.iter('MedlineCitation'):
            
            # 获取pmid
            pmid = citation.find('PMID')
            if pmid == None:
                continue
            else:
                pmid = pmid.text
            
            #获取标题
            texts = {}
            title = citation.find('Article/ArticleTitle')
            if title is None:
                texts["Title"] = ""
            else:
                texts["Title"] = " ".join(title.itertext())
            
            # 获取期刊
            journal = citation.find("Article/Journal/Title")
            if journal is None:
                texts["Journal"] = ""
            else:
                texts["Journal"] = " ".join(journal.itertext())
                
            # 获取摘要，摘要存在分段
            info_l = []
            for info in citation.iter('AbstractText'):
                if info is None:
                    abstract = ""
                if info is not None:
                    abstract = " ".join(info.itertext())
                    info_l.append(abstract)
            texts["Ab"] = texts["Title"] + " " + " ".join(info_l)
            
            # 存取mesh词汇
            MHs = []
            mesh = citation.find("MeshHeadingList")
            if  mesh is None:
                MHs = []
            else:
                for MH in mesh:
                    MHs.append([mh.text for mh in MH])

            # 存取日期
            date = citation.find("DateCompleted/Year")
            if  date is None:
                dates = ""
            else:
                dates = date.text
            
            output[pmid] = {'pmid': pmid,
                            'texts': texts,
                            'Meshhead':MHs,
                            "date":dates}
        
        with open('pubmed_2022d/%s.json' % xml_path.split('.')[0], 'w') as f:
            json.dump(output, f, indent=4)
            f.close()

# 合并

In [ ]:
# 大文件
import os
import glob
import json

files = os.listdir("pubmed_2022d")
files = sorted(files)
papers_d = {}

if os.path.exists("pubmed_2022_merge/pubmed_done.json") is False:
    done_l = []
    print("done_l is empty")
else:
    with open("pubmed_2022_merge/pubmed_done.json") as f:
        done_l = json.load(f)
        print("done_l is exists")
files = [i for i in files if i not in done_l]

for i, file in enumerate(files):
    
    with open("pubmed_2022d/%s" % file, "r") as f:
        paper_d = json.load(f)
    print(file)
    
    papers_d.update(paper_d)
    done_l.append(file)
    
    iter_list = set(range(99, 1114, 100))
    iter_list.add(1114)
    if i in iter_list:
        with open("pubmed_2022_merge/pubmed_%s.txt" % len(done_l), "w") as f:
            b = json.dumps(papers_d)
            f.write(b)
        with open("pubmed_2022_merge/pubmed_done.json" , "w") as f:
            json.dump(done_l,f)
        papers_d = {}